In [13]:
import pandas as pd
df = pd.read_csv('/home/dkovac/PycharmProjects/Proteus/data/eval.csv',
                        delimiter=',', header=0, quotechar='"')

df['section_names'] = df['section_names'].str.lower()
df['apis'] = df['apis'].str.lower()
df['imports'] = df['imports'].str.lower()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6928 entries, 0 to 6927
Data columns (total 33 columns):
Unnamed: 0                          6928 non-null int64
sha1                                6928 non-null object
entropy                             6928 non-null float64
imports                             6928 non-null object
imports_count                       6928 non-null int64
apis                                6928 non-null object
apis_count                          6928 non-null int64
has_children                        6928 non-null bool
children_entropies_mean             4290 non-null float64
children_entropies_std              4290 non-null float64
section_names                       6926 non-null object
text_entropy                        3785 non-null float64
data_entropy                        3280 non-null float64
rdata_entropy                       2073 non-null float64
buildid_entropy                     244 non-null float64
pdata_entropy                       2

In [9]:
from sklearn.base import BaseEstimator, TransformerMixin


class ItemSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key, dt):
        self.key = key
        self.dt = dt

    def fit(self, x, y=None):
        # does nothing
        return self

    def transform(self, data_dict):
        # this returns the column requested
        if self.dt == 'text':
            return data_dict.loc[:, self.key]
        elif self.dt == 'num2text':
            return data_dict.loc[:, self.key].astype(unicode)
        elif self.dt == 'date':
            return (data_dict.loc[:, self.key].squeeze() - pd.Timestamp(1900, 1, 1)).dt.days.to_frame()
        else:
            return data_dict.loc[:, [self.key]].astype(float)



In [10]:
from sklearn.externals import joblib
import csv 


model = joblib.load('/home/dkovac/PycharmProjects/Proteus/model/model.pkl')
df['prediction'] = model.predict(df)


/home/dkovac/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [4]:
with open('/home/dkovac/PycharmProjects/Proteus/data/order_list.txt') as f:
    order_list = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
order_list = [x.strip() for x in order_list] 
print(order_list)


/home/dkovac/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [5]:

df = df.set_index('sha1')
df.loc[order_list]



/home/dkovac/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [12]:

results = df[['sha1', 'prediction']]
results.to_csv('/home/dkovac/PycharmProjects/Proteus/data/eval1.tsv', sep='\t', index=True, header=False, quoting=csv.QUOTE_NONE)
